# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.5 MB 14.3 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.3
    Uninstalling pandas-1.4.3:
      Successfully uninstalled pandas-1.4.3
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 31.7 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.20.1
    Uninstalling transformers-4.20.1:
      Successfully uninstalled transformers-4.20.1
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 290 kB 17.4 MB/s eta 0:00:01
  

In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Global variables

In [4]:
dataset_df = pd.read_pickle("/notebooks/token_classification/data/pe_dataset_task_1_bio_only_w_ints_correction_pickle")

In [5]:
dataset_df

,split,essay,essay_nr,para_nr,paragraph,paragraph_words,paragraph_tags,tags_list_int,paragraph_tags_bio_only,paragraph_tags_bio_only_int,paragraph_tags_bio_only_correction
0,TRAIN,Should students be taught to compete or to coo...,essay001,1,It is always said that competition can effecti...,"[It, is, always, said, that, competition, can,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,TRAIN,Should students be taught to compete or to coo...,essay001,2,"First of all, through cooperation, children ca...","[First, of, all, ,, through, cooperation, ,, c...","[O, O, O, O, B-Claim, I-Claim, I-Claim, I-Clai...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[O, O, O, O, B, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I..."
2,TRAIN,Should students be taught to compete or to coo...,essay001,3,"On the other hand, the significance of competi...","[On, the, other, hand, ,, the, significance, o...","[O, O, O, O, O, B-Premise, I-Premise, I-Premis...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP..."
3,TRAIN,Should students be taught to compete or to coo...,essay001,4,"Consequently, no matter from the view of indiv...","[Consequently, ,, no, matter, from, the, view,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,TRAIN,More people are migrating to other countries t...,essay002,1,The last 50 years have seen an increasing numb...,"[The, last, 50, years, have, seen, an, increas...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...,...,...,...,...
1714,TRAIN,Fatherhood should be as present as motherhood ...,essay401,4,"In conclusion, fatherhood should be as present...","[In, conclusion, ,, fatherhood, should, be, as...","[O, O, O, B-MajorClaim, I-MajorClaim, I-MajorC...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B, I, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I-CO..."
1715,TRAIN,Children should studying hard or playing sport...,essay402,1,Some people believe that studying hard is esse...,"[Some, people, believe, that, studying, hard, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1716,TRAIN,Children should studying hard or playing sport...,essay402,2,"On the other hand, studying hard will give chi...","[On, the, other, hand, ,, studying, hard, will...","[O, O, O, O, O, B-Claim, I-Claim, I-Claim, I-C...","[0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 5, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, O, B, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP..."
1717,TRAIN,Children should studying hard or playing sport...,essay402,3,"At the same time, playing sports will give goo...","[At, the, same, time, ,, playing, sports, will...","[O, O, O, O, O, B-Claim, I-Claim, I-Claim, I-C...","[0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, ...","[O, O, O, O, O, B, I, I, I, I, I

In [6]:
dataset_df.columns

Index(['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words',
       'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only',
       'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
      dtype='object')

In [7]:
# sanity check
print(len(dataset_df))
dataset_df = dataset_df.dropna()
print(len(dataset_df))

1719
1719


In [8]:
dataset_df = dataset_df.reset_index(drop=True)

In [9]:
dataset_df

,split,essay,essay_nr,para_nr,paragraph,paragraph_words,paragraph_tags,tags_list_int,paragraph_tags_bio_only,paragraph_tags_bio_only_int,paragraph_tags_bio_only_correction
0,TRAIN,Should students be taught to compete or to coo...,essay001,1,It is always said that competition can effecti...,"[It, is, always, said, that, competition, can,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,TRAIN,Should students be taught to compete or to coo...,essay001,2,"First of all, through cooperation, children ca...","[First, of, all, ,, through, cooperation, ,, c...","[O, O, O, O, B-Claim, I-Claim, I-Claim, I-Clai...","[0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[O, O, O, O, B, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I..."
2,TRAIN,Should students be taught to compete or to coo...,essay001,3,"On the other hand, the significance of competi...","[On, the, other, hand, ,, the, significance, o...","[O, O, O, O, O, B-Premise, I-Premise, I-Premis...","[0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP..."
3,TRAIN,Should students be taught to compete or to coo...,essay001,4,"Consequently, no matter from the view of indiv...","[Consequently, ,, no, matter, from, the, view,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,TRAIN,More people are migrating to other countries t...,essay002,1,The last 50 years have seen an increasing numb...,"[The, last, 50, years, have, seen, an, increas...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...,...,...,...,...,...,...,...,...
1714,TRAIN,Fatherhood should be as present as motherhood ...,essay401,4,"In conclusion, fatherhood should be as present...","[In, conclusion, ,, fatherhood, should, be, as...","[O, O, O, B-MajorClaim, I-MajorClaim, I-MajorC...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B, I, I, I, I, I, I, I, I, I, I, I, ...","[0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[O, O, O, B-COMP, I-COMP, I-COMP, I-COMP, I-CO..."
1715,TRAIN,Children should studying hard or playing sport...,essay402,1,Some people believe that studying hard is esse...,"[Some, people, believe, that, studying, hard, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1716,TRAIN,Children should studying hard or playing sport...,essay402,2,"On the other hand, studying hard will give chi...","[On, the, other, hand, ,, studying, hard, will...","[O, O, O, O, O, B-Claim, I-Claim, I-Claim, I-C...","[0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 5, ...","[O, O, O, O, O, B, I, I, I, I, I, I, I, O, B, ...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, ...","[O, O, O, O, O, B-COMP, I-COMP, I-COMP, I-COMP..."
1717,TRAIN,Children should studying hard or playing sport...,essay402,3,"At the same time, playing sports will give goo...","[At, the, same, time, ,, playing, sports, will...","[O, O, O, O, O, B-Claim, I-Claim, I-Claim, I-C...","[0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, ...","[O, O, O, O, O, B, I, I, I, I, I

In [10]:
train_df = dataset_df[dataset_df.split=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split=='TEST'].reset_index(drop=True)



# for example

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)

## Prepare Dataset object

In [11]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [12]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [13]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 273
    })
})

In [15]:
# sanity check
set(dataset['train']['split']), set(dataset['test']['split']), set(dataset['validation']['split'])

({'TRAIN'}, {'TEST'}, {'TRAIN'})

## Save dataset

In [16]:
torch.save(dataset, os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'))

In [17]:
with open(os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'), "wb") as f:
    pickle.dump(dataset, f)
#torch.save(dataset, os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'))